In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [5]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [6]:
y_val = housing['medianHouseValue']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

In [11]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [13]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [14]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [15]:
import tensorflow as tf

In [16]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [17]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [18]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [20]:
model = tf.contrib.learn.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)
# model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_task_type': None, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000225999DD358>, '_is_chief': True, '_num_worker_replicas': 0, '_environment': 'local', '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_save_checkpoints_steps': None, '_model_dir': 'C:\\Users\\asus\\AppData\\Local\\Temp\\tmp3j_j27_t', '_task_id': 0, '_master': '', '_session_config': None}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [22]:
model.fit(input_fn=input_func,steps=25000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\asus\AppData\Local\Temp\tmp3j_j27_t\model.ckpt.
INFO:tensorflow:step = 1, loss = 4.74811e+10
INFO:tensorflow:global_step/sec: 321.068
INFO:tensorflow:step = 101, loss = 4.64502e+10 (0.324 sec)
INFO:tensorflow:global_step/sec: 278.087
INFO:tensorflow:step = 201, loss = 3.31106e+10 (0.357 sec)
INFO:tensorflow:global_step/sec: 296.491
INFO:tensorflow:step = 301, loss = 5.50451e+10 (0.336 sec)
INFO:tensorflow:global_step/sec: 315.513
INFO:tensorflow:step = 401, loss = 8.40013e+10 (0.322 sec)
INFO:tensorflow:global_step/sec: 287.197
INFO:tensorflow:step = 501, loss = 6.3

INFO:tensorflow:global_step/sec: 355.579
INFO:tensorflow:step = 7501, loss = 1.35365e+10 (0.280 sec)
INFO:tensorflow:global_step/sec: 306.384
INFO:tensorflow:step = 7601, loss = 1.5071e+10 (0.321 sec)
INFO:tensorflow:global_step/sec: 321.552
INFO:tensorflow:step = 7701, loss = 5.65676e+09 (0.313 sec)
INFO:tensorflow:global_step/sec: 339.986
INFO:tensorflow:step = 7801, loss = 6.90869e+09 (0.295 sec)
INFO:tensorflow:global_step/sec: 325.665
INFO:tensorflow:step = 7901, loss = 1.74628e+10 (0.308 sec)
INFO:tensorflow:global_step/sec: 303.214
INFO:tensorflow:step = 8001, loss = 7.64263e+09 (0.333 sec)
INFO:tensorflow:global_step/sec: 305.881
INFO:tensorflow:step = 8101, loss = 1.13467e+10 (0.322 sec)
INFO:tensorflow:global_step/sec: 305.012
INFO:tensorflow:step = 8201, loss = 8.11516e+09 (0.328 sec)
INFO:tensorflow:global_step/sec: 320.999
INFO:tensorflow:step = 8301, loss = 1.11403e+10 (0.314 sec)
INFO:tensorflow:global_step/sec: 319.515
INFO:tensorflow:step = 8401, loss = 1.36739e+10 (0.

INFO:tensorflow:global_step/sec: 322.58
INFO:tensorflow:step = 15601, loss = 1.25983e+10 (0.305 sec)
INFO:tensorflow:global_step/sec: 304.092
INFO:tensorflow:step = 15701, loss = 1.75052e+10 (0.328 sec)
INFO:tensorflow:global_step/sec: 317.175
INFO:tensorflow:step = 15801, loss = 6.59659e+09 (0.319 sec)
INFO:tensorflow:global_step/sec: 306.873
INFO:tensorflow:step = 15901, loss = 1.14775e+10 (0.324 sec)
INFO:tensorflow:global_step/sec: 312.806
INFO:tensorflow:step = 16001, loss = 1.47489e+10 (0.320 sec)
INFO:tensorflow:global_step/sec: 293.893
INFO:tensorflow:step = 16101, loss = 4.17067e+09 (0.338 sec)
INFO:tensorflow:global_step/sec: 325.511
INFO:tensorflow:step = 16201, loss = 1.38122e+10 (0.309 sec)
INFO:tensorflow:global_step/sec: 296.927
INFO:tensorflow:step = 16301, loss = 1.12102e+10 (0.339 sec)
INFO:tensorflow:global_step/sec: 297.364
INFO:tensorflow:step = 16401, loss = 1.29888e+10 (0.334 sec)
INFO:tensorflow:global_step/sec: 294.395
INFO:tensorflow:step = 16501, loss = 1.170

INFO:tensorflow:global_step/sec: 316.008
INFO:tensorflow:step = 23701, loss = 1.59078e+10 (0.310 sec)
INFO:tensorflow:global_step/sec: 341.716
INFO:tensorflow:step = 23801, loss = 9.85984e+09 (0.298 sec)
INFO:tensorflow:global_step/sec: 322.066
INFO:tensorflow:step = 23901, loss = 7.0714e+09 (0.313 sec)
INFO:tensorflow:global_step/sec: 292.192
INFO:tensorflow:step = 24001, loss = 7.72781e+09 (0.336 sec)
INFO:tensorflow:global_step/sec: 324.616
INFO:tensorflow:step = 24101, loss = 3.21969e+09 (0.314 sec)
INFO:tensorflow:global_step/sec: 305.049
INFO:tensorflow:step = 24201, loss = 1.2307e+10 (0.321 sec)
INFO:tensorflow:global_step/sec: 344.05
INFO:tensorflow:step = 24301, loss = 8.30207e+09 (0.293 sec)
INFO:tensorflow:global_step/sec: 308.239
INFO:tensorflow:step = 24401, loss = 6.92505e+09 (0.322 sec)
INFO:tensorflow:global_step/sec: 332.738
INFO:tensorflow:step = 24501, loss = 2.45113e+09 (0.303 sec)
INFO:tensorflow:global_step/sec: 329.432
INFO:tensorflow:step = 24601, loss = 1.10761

DNNRegressor(params={'embedding_lr_multipliers': None, 'gradient_clip_norm': None, 'input_layer_min_slice_size': None, 'feature_columns': (_NumericColumn(key='housingMedianAge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='totalRooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='totalBedrooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='households', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='medianIncome', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)), 'activation_fn': <function relu at 0x000002259D4AB7B8>, 'optimizer': None, 'hidden_units': [6, 6, 6], 'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x00000225999DD9E8>, 'dropout': No

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [23]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [24]:
pred_gen = model.predict(predict_input_func)

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


AttributeError: 'function' object has no attribute 'dtype'

In [155]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from C:\Users\Marcial\AppData\Local\Temp\tmp244d0d44\model.ckpt-25000


** Calculate the RMSE. Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [156]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [157]:
from sklearn.metrics import mean_squared_error

In [158]:
mean_squared_error(y_test,final_preds)**0.5

97921.93181985477